<a href="https://colab.research.google.com/github/dSatasiya/WLC-With-Machine-Learning/blob/main/2_bit_system_DNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np                      # to perform the various numerical operation
import random                           # to generate the random values for the matrix
import torch                            # PyTorch - machine learning framwork 
from torch import nn                    # Neural Network sublibrary from torch
import torch.optim as optim             # Optimizer to optimiz the neural network to reduce the loss
import math                             # to use inbuilt mathematical formulas
from matplotlib import pyplot as plt    # to plot the final output 
import tensorflow as tf

In [ ]:
bit = 2
SNR_db = np.array(list(range(2, 11, 2)))
print(f" Different SNR values : {SNR_db}")
batch_size = 1

 Different SNR values : [ 2  4  6  8 10]


In [ ]:
def generate_data_bit(bit, batch_size):

  input = [[0 for _ in range(pow(2, bit))] for _ in range(bit)]
  label = [[0 for _ in range(pow(2, bit))] for _ in range(2 * bit)]

  for i in range(1<<bit):       # 1<<2 = 1 = 0001 and 1<<2 = 0100 = 4 in decimal 
    for j in range(bit):        # bit = 2 = (0 and 1)
      if i & (1<<j):            # & = AND operation 
        input[bit - j -1][i] = 1

  for i in range(1<<bit):
    for j in range(bit):
      if input[j][i] == 1:
        label[2*j][i] = 1
        label[2*j+1][i] = 0
      else:
        label[2*j][i] = 0
        label[2*j+1][i] = 1

  input = np.tile(np.array(input) ,batch_size)  # to repeat the array to produce the multiple inputs 
  label = np.tile(np.array(label) ,batch_size)  # same for labels
  # input_s = input.shape
  # label_s = label.shape
  return input, label # input_s, label_s
generate_data_bit(bit, batch_size)


(array([[0, 0, 1, 1],
        [0, 1, 0, 1]]), array([[0, 0, 1, 1],
        [1, 1, 0, 0],
        [0, 1, 0, 1],
        [1, 0, 1, 0]]))

In [ ]:
def generate(M, N, batch_size):                        # What are the M and N ?
  input, label = generate_data_bit(M, batch_size)
  ran1 = np.random.permutation(batch_size * pow(2, M))  # Suffling Dataset
  ran2 = np.random.permutation(batch_size * pow(2, M))
  
  symbol1 = 2 * input[:, ran1] - 1                  # BPSK is applied over here
  symbol2 = 2 * input[:, ran2] - 1  # [rows, columns]
  # signal1 = 
  # SPC = math.sqrt(0.8) * symbol1 + math.sqrt(0.2) * symbol2  # Superposition Coding

  label1 = np.transpose(label[:, ran1].astype('float32')) # Transpose of matrix
  label2 = np.transpose(label[:, ran2].astype('float32'))
  return SPC, label1, label2, symbol1, symbol2  #SPC.shape, label1.shape, label2.shape

generate(2, 2, 1) 

(array([[ 1.34164079,  0.4472136 , -0.4472136 , -1.34164079],
        [-1.34164079,  0.4472136 ,  1.34164079, -0.4472136 ]]),
 array([[1., 0., 0., 1.],
        [1., 0., 1., 0.],
        [0., 1., 1., 0.],
        [0., 1., 0., 1.]], dtype=float32),
 array([[1., 0., 0., 1.],
        [0., 1., 0., 1.],
        [1., 0., 1., 0.],
        [0., 1., 1., 0.]], dtype=float32),
 array([[ 1,  1, -1, -1],
        [-1,  1,  1, -1]]),
 array([[ 1, -1,  1, -1],
        [-1, -1,  1,  1]]))

In [ ]:
# def generate_input(H1_real, H1_image, SPC, N, batch_size, sigma):   # What is "Sigma" here ?
def generate_input(H1_real, H1_image, SPC, N, batch_size, sigma): 
    N_real, N_image = generate_channel(N, batch_size * pow(2, N), 0)

    input1_real = np.matmul(H1_real, SPC) + sigma * N_real       # Matrix multiplications  and the value of sigma will be constant
    input1_img = np.matmul(H1_image, SPC) + sigma * N_image         # and overall value of input will be in the form of matrix 

    # I must emphasize that in order to add or subtract two given matrices, they should have the same size or dimension. 
    # Otherwise, we conclude that the sum (addition) or difference (subtraction) of two matrices having different sizes or dimensions is undefined!

    input1 = np.transpose(np.concatenate((input1_real, input1_img), axis=0))  # axis = 0 means vertical joining and axis = 1 means horizontal joining
                                                                 # concatenate in used to join the arrays in the particular sequence
    return input1
# generate_input()

(array([[ 0.05174115,  0.4973521 ],
        [ 0.56055852, -0.50396289]]), array([[ 0.84469894, -0.0208816 ],
        [ 0.10053351, -0.63961375]]))

In [ ]:
def generate_channel(N, M, k):

  h1 = np.random.randn(N, M)/ math.sqrt(2)  # to create arbitrary signals,
  h2 = np.random.randn(N, M)/ math.sqrt(2)  # we use random.randn to create random matrics 

  if k == 0:
    return h1, h2
  else:
    return 2 * h1, 2 * h2
generate_channel(2, 2, 0)

(array([[-0.35175591,  0.11070654],
        [ 0.1988587 ,  0.8149869 ]]), array([[ 1.2537708 ,  0.23677384],
        [-0.35053621, -0.1996871 ]]))

In [ ]:
def accuracy(out, label, bit):
  accuracy = 0
  for i in range(bit):

     acc =tf.reduce_mean(tf.cast(tf.equal(tf.argmax(tf.slice(out, [0, 2*i], [-1, 2]), 1), tf.argmax(tf.slice(label, [0, 2*i], [-1, 2]), 1)), dtype=tf.float32))
     accuracy += acc
  return accuracy / 2

1) **tf.slice(input_, begin_, size, name=none)** is basically used to cut or slice the array from particular starting point with typical size.

2) tf.argmax(input_, name) is used to find the position of maximum value in the array and it will give the position of max value rowwise

3) tf.equal(x, y)  --> Returns the truth value of (x == y) element-wise.

4) tf.cast(x, dtype) --> Casts a tensor(x here) to a new type

5) tf.reduce_mean(input_tensor) --> Computes the mean of elements across dimensions of a tensor. (here for input_tensor)

Here, fisrt of all accuracy of the half vertical array will be found and then the accuracy of another half vertical array will be found. 

In [ ]:
net = nn.Sequential(
    nn.Linear(4, 8), nn.BatchNorm1d(8), nn.ReLU(),    
    nn.Linear(8, 8), nn.BatchNorm1d(8), nn.ReLU(),
    # nn.Linear(8, 8), nn.BatchNorm1d(8), nn.ReLU(),
    # nn.Linear(8, 8), nn.BatchNorm1d(8), nn.ReLU(),
    # nn.Linear(8, 8), nn.BatchNorm1d(8), nn.ReLU(),    # Because, Activation funtion will only run at output side 
    nn.Linear(8, 4), nn.BatchNorm1d(4),               # So, we haven't written it here
)
# print(net)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
loss = nn.MSELoss()        # it is used to find the Mean Square Error Loss in the system 
net1 = net.to(device = device)     # net1 for user1 
net2 = net.to(device = device)     # net2 for user2 
# print(net1)
# print(net2)
def init_weights(m):                                  # what does this mean ??
    if type(m) == nn.Linear:                          # this somewhat needs to understand
        torch.nn.init.normal_(m.weight, std=0.01)     #Fills the input Tensor with values drawn from the normal distribution 
init_weights(2)
net1.apply(init_weights)
net2.apply(init_weights)

Sequential(
  (0): Linear(in_features=4, out_features=8, bias=True)
  (1): BatchNorm1d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): Linear(in_features=8, out_features=8, bias=True)
  (4): BatchNorm1d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (5): ReLU()
  (6): Linear(in_features=8, out_features=4, bias=True)
  (7): BatchNorm1d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [ ]:
H1_real, H1_image = generate_channel(bit, bit, 0)
H2_real, H2_image = generate_channel(bit, bit, 1)
batch_size = 1
train_size = 1
N = 2
# print(H1_real, H1_image)
# print(H2_real, H2_image) 
cnt = 0       # count is initially set to 0 
iter = 10    # iteration is 5 nothing but epoch for the neural network 
print("---------------------------------Training Operation---------------------------------")
for i in range(len(SNR_db)):            # print(i)                   
  SPC, label1_train, _ = generate(bit, bit, 1)
  signal_power = np.mean(pow(SPC, 2))
  sigma_user1  = math.sqrt(signal_power / (math.sqrt(bit) * pow(10, float(SNR_db[i]) / 10.0)))   # this is just formula applied for the value of sigma
  input1_train = generate_input(H1_real, H1_image, SPC, N, batch_size, sigma_user1)
  input1 = generate_input(H1_real, H1_image, SPC, N, batch_size, sigma_user1)
  input1 = torch.Tensor(input1)   # input1 is converted from numpy array to tensor
  label1 = label1_train
  label1 = torch.Tensor(label1) 
# ----------------------------------------- Extra Lines -------------------------------------------
  # H1_real, H1_image = generate_channel(2, 2, 0)
  # N_real, N_image = generate_channel(N, batch_size * pow(2, N), 0)
  # input1_real, _= generate_input(H1_real, H1_image, SPC, N, batch_size, sigma_user1)
  # _, input1_img = generate_input(H1_real, H1_image, SPC, N, batch_size, sigma_user1)

  # input1 = input1.view(-1, 4*4)       #  It will simply reshape the entire tensor in unknown dimension
# --------------------------------------------------------------------------------------------------

  SPC, _, label2_train = generate(bit, bit, 1)
  signal_power = np.mean(pow(SPC, 2))
  sigma_user2 = math.sqrt(signal_power / (math.sqrt(bit) * pow(10, float(SNR_db[i]) / 10.0)))
  input2_train = generate_input(H2_real, H2_image, SPC, N, batch_size, sigma_user2)
  input2 = input2_train
  input2 = torch.Tensor(input2)
  label2 = label2_train
  label2 = torch.Tensor(label2)
# ---------------------------------- extra -----------------------------------------
  # input1_train = torch.Tensor(input1_train) 
  # label1_train = torch.Tensor(label1_train)zz
  # input1 = input1_train.cuda() if torch.cuda.is_available() else 'cpu'      # string error is probably arising due to "cuda"
  # label1= label1_train.cuda() if torch.cuda.is_available() else 'cpu'
# ----------------------------------------------------------------------------------
  for j in range(iter):
    # print(f"This is iteration -- {j}")
    optimizer1 = optim.SGD(net1.parameters(), lr = 0.01)          # The fundamental goal of optimizer is to reduce the loss and to decrease
    optimizer1.zero_grad()
    # print(f"input ---- 1 ---- \n {input1}")                                      # the training error in the neural network 
    output1 = net1(input1)                         # here 
    # print(f"output ---- 1 ---- \n {output1}")
    # print(f"label ---- 1 ---- \n {label1}")    
    l1 = loss(output1, label1)
    # print(f"loss ---- 1 ---- \n {l1}")
    l1.backward()
    optimizer1.step()
    op1 = torch.Tensor.cpu(output1)
    out1 = op1.detach().numpy()
    # print(f"Numpy array ---- 1 ---- \n {out1}")                           # detach(). numpy() is the correct way to get a numpy array from a torch tensor.
    acc1 = accuracy(out1, label1, bit) 
    cnt = cnt+1

    output2 = net2(input2)
    # print(output2[0][0])
    optimizer2 = optim.SGD(net2.parameters(), lr = 0.01)
    optimizer2.zero_grad()
    l2 = loss(output2, label2)
    l2.backward()
    optimizer2.step()  
    op2 = torch.Tensor.cpu(output2)
    out2 = op2.detach().numpy()
    acc2 = accuracy(out2, label2, bit)
    print("iter {} User1 aacuracy : {} User2 accuracy : {}".format(cnt,acc1,acc2))
  print("-------------------------------------------------")

# ----------------------------- Extra Lines to check the outputs --------------------------
# print(label1)
# print(label1_train.shape)
# print(SPC)
# print(f"Signal Power is : {signal_power}")
# print(f"The value of Sigma for User-1 is : {sigma_user1}")
# print(input1_train)
# print(input1_train.shape)

# print(input1_real)
# print(input1_real.shape) 
# print(input1_img)
# print(input1_img.shape) 

# input1 = torch.set_printoptions(profile= "full")
# input1 = torch.set_printoptions(profile= "default")
# print(f"---------Input to the nueral network -------- \n {input1}")
# print(input1.shape)
# output1 = torch.set_printoptions(profile= "full")
# output1 = torch.set_printoptions(profile= "default")
# print(f"---------Output from the nueral network -------- \n {output1}")
# print(output1.shape)

# print(H1_real)
# print(H1_real.shape)
# print(H1_image)
# print(H1_image.shape)
# print(N_real)
# print(N_real.shape)
# print(N_image)
# print(N_image.shape)

# print(label2_train)
# print(f"Signal Power is : {signal_power}")
# print(f"The value of Sigma for User-2 is : {sigma_user2}")
# print(input2_train)
# print(label2_train.shape)
# print(input2_train.shape)

# print(bit)

# print(f"Accuracy Array -------- \n {acc1}")
#-------------------------------------------------------------------------------------------

---------------------------------Training Operation---------------------------------
iter 1 User1 aacuracy : 0.375 User2 accuracy : 0.75
iter 2 User1 aacuracy : 0.25 User2 accuracy : 0.25
iter 3 User1 aacuracy : 0.25 User2 accuracy : 0.875
iter 4 User1 aacuracy : 0.25 User2 accuracy : 0.875
iter 5 User1 aacuracy : 0.375 User2 accuracy : 0.625
iter 6 User1 aacuracy : 0.5 User2 accuracy : 0.625
iter 7 User1 aacuracy : 0.5 User2 accuracy : 0.75
iter 8 User1 aacuracy : 0.75 User2 accuracy : 0.75
iter 9 User1 aacuracy : 0.875 User2 accuracy : 0.75
iter 10 User1 aacuracy : 0.75 User2 accuracy : 0.875
-------------------------------------------------
iter 11 User1 aacuracy : 0.75 User2 accuracy : 0.5
iter 12 User1 aacuracy : 0.5 User2 accuracy : 0.625
iter 13 User1 aacuracy : 0.75 User2 accuracy : 0.75
iter 14 User1 aacuracy : 0.625 User2 accuracy : 0.875
iter 15 User1 aacuracy : 0.625 User2 accuracy : 0.875
iter 16 User1 aacuracy : 0.5 User2 accuracy : 0.875
iter 17 User1 aacuracy : 0.625 Us

In [ ]:
Train = { "input1_train" : input1_train, "train_label1" : label1_train, "input2_train" : input2_train, "train_label2" : label2_train }
print("---------------Training data -------------- \n")
print('User1 train input : {}'.format(input1_train.shape))
print('User1 train label : {}'.format(label1_train.shape))
print('User2 train input : {}'.format(input2_train.shape))
print('User2 train label : {}'.format(label2_train.shape))

---------------Training data -------------- 

User1 train input : (4, 4)
User1 train label : (4, 4)
User2 train input : (4, 4)
User2 train label : (4, 4)


In [ ]:
ERROR_user1 = np.zeros([len(SNR_db), 1])
ERROR_user2 = np.zeros([len(SNR_db), 1])
print(ERROR_user1)
print(ERROR_user2)


[[0.]
 [0.]
 [0.]
 [0.]
 [0.]]
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]]


In [ ]:
print('---------------------------------Testing operation-------------------------')
test_size = 1            # not previously defined 
batch_size = 1
k = 0
for i in range(len(SNR_db)):
  SPC_test, label1_test, label2_test = generate(bit, bit, batch_size * test_size)
  signal_power = np.mean(pow(SPC_test, 2))
  sigma_test = math.sqrt(signal_power / (math.sqrt(bit) * pow(10, float(SNR_db[i]) / 10.0)))
  input1_test = generate_input(H1_real, H1_image, SPC_test, bit, batch_size * test_size, sigma_test)
  input2_test = generate_input(H2_real, H2_image, SPC_test, bit, batch_size * test_size, sigma_test)
      
  input1_test = torch.Tensor(input1_test)
  label1_test = torch.Tensor(label1_test)
  # input_test1 = input1_test.cuda()        # Changes in cuda 
  # label_test1 = label1_test.cuda()
      
  testop1 = net1(input1_test)
  opt1 = torch.Tensor.cpu(testop1)
  out1 = opt1.detach().numpy()
  ac1 = accuracy(out1, label1_test, bit)
  # print(ac1)
  ERROR_user1[i, k] = 1 - ac1

  input2_test = torch.Tensor(input2_test)
  label2_test = torch.Tensor(label2_test)
  # input_test2 = input2_test.cuda()
  # label_test2 = label2_test.cuda()
      
  testop2 = net2(input2_test)
  opt2 = torch.Tensor.cpu(testop2)
  out2 = opt2.detach().numpy()
  ac2 = accuracy(out2, label2_test, bit)
  # print(ac2)
  ERROR_user2[i, k] = 1 - ac2

  # -------------------------------------------Extra Lines of the code ----------------------------------
# print(label1_test)
# print(label2_test)
print(f"Error for user-1 for 5 different values of SNR -- \n {ERROR_user1}")
print(f"Error for user-2 for 5 different values of SNR -- \n {ERROR_user2}")
# print(SPC_test)

---------------------------------Testing operation-------------------------
Error for user-1 for 5 different values of SNR -- 
 [[0.   ]
 [0.125]
 [0.   ]
 [0.125]
 [0.   ]]
Error for user-2 for 5 different values of SNR -- 
 [[0.375]
 [0.   ]
 [0.25 ]
 [0.625]
 [0.5  ]]


In [ ]:
Test = { "input1_test" : input1_test, "input2_test" : input2_test, "test_label1" : label1_test, "test_label2" : label2_test }
print("----------------Testing data-------------------")

print('User1 test input : {}'.format(input1_test.shape))
print('User1 test label : {}'.format(label1_test.shape))
print('User2 test input : {}'.format(input2_test.shape))
print('User2 test label : {}'.format(label2_test.shape))

----------------Testing data-------------------
User1 test input : torch.Size([4, 4])
User1 test label : torch.Size([4, 4])
User2 test input : torch.Size([4, 4])
User2 test label : torch.Size([4, 4])


1) plt.semilog => :Make a plot with log scaling on the y axis.
2) plt.annotate()
matplotlib.pyplot.annotate(text, xy, *args, **kwargs)
Annotate the point xy with text text. (here in our case text = label1/label2)
 
3) plt.grid()
Configure the grid lines. 
example - grid(color='r', linestyle='-', linewidth=2)

4) plt.legend() 
The call signatures correspond to the following different ways to use this method

5) 

In [ ]:
error1 = np.mean(ERROR_user1, axis=1)       # axis = 1 means horizontal joining 
error2 = np.mean((ERROR_user2), axis=1)
# print(error1)
# print(error2)
plt.figure()
plt.semilogy(SNR_db, error1, ls='--', marker='o', label='user1')        # where "ls" = line style
plt.semilogy(SNR_db, error2, ls='--', marker='+', label='user2')
for i in range(5):
  label1 = "{:.2f}".format(error1[i])
  label2 = "{:.2f}".format(error2[i])
  plt.annotate(label1,(SNR_db[i],ERROR_user1[i]))
  plt.annotate(label2,(SNR_db[i],ERROR_user2[i]))
plt.grid()
plt.legend()
plt.xlim(pow(10, 0), pow(11, 1))
plt.xlabel('SNR')
plt.ylabel('SER')
plt.title('SER of user2 in MIMO_NOMA BPSK_DNN')
plt.savefig('SER_MIMO_NOMA_DNN_BPSK')
plt.show()
print("SNR : {}".format(SNR_db))
print("bit error for user 1 at testing : {}".format(error1))
print("bit error for user 2 at testing : {}".format(error2))

ValueError: ignored

<Figure size 432x288 with 1 Axes>

SNR : [ 2  4  6  8 10]
bit error for user 1 at testing : [0.    0.125 0.    0.125 0.   ]
bit error for user 2 at testing : [0.375 0.    0.25  0.625 0.5  ]
